**Exclusions:**
Exclude generic, or non-specific, or emotive affirmations that do not add substantial or verifiable information about the policy:
```
Does the privacy policy affirm that the company respects user privacy?
Does the privacy policy affirm that the company is strongly committed to keeping secure any information obtained from or about users?
```

In [ ]:
from policy_analyser import AnalysisProcessor

data_source = {
	"openai": "https://openai.com/policies/privacy-policy/",
	# "anthropic": "https://www.anthropic.com/legal/privacy",
	# "perplexity": "https://www.perplexity.ai/hub/legal/privacy-policy",	# HTTPError: 403 Client Error: Forbidden for url: https://www.perplexity.ai/hub/legal/privacy-policy
	# "deepseek": "https://cdn.deepseek.com/policies/en-US/deepseek-privacy-policy.html", # Returns None
}

analyser = AnalysisProcessor(data_source, substring_mode=1)
analyser.runAnalyses(limit_iterations=4)

KeyboardInterrupt: 

In [34]:
from model_management import GeminiModel
from policy_analyser import loadQuestions, loadPolicies
import numpy as np
from google.genai import types

q_dict = loadQuestions()
questions = list(q_dict.keys())[:20]
m = GeminiModel()
alt = [GeminiModel.preprocessStatement_alt(q) for q in questions]

_embeddings = m.client.models.embed_content(
	model="gemini-embedding-001",
	contents=alt,
	config=types.EmbedContentConfig(task_type="FACT_VERIFICATION"),
)

In [35]:
q_embeddings = [e.values for e in _embeddings.embeddings]
q_embeddings
pol = loadPolicies()["openai"][
	"64b194a739dade62a6c90a81d803b42e298c2dcccb79a1f1fcca83df888ad67c"
]["policy_chunks"]

p_embeddings = np.stack([p["retreval_embedding_vector"] for p in pol])


# print(f"q_embeddings: {type(q_embeddings[0])}, {q_embeddings[0]}")
# print(f"q_embeddings: {type(p_embeddings[0])}, {p_embeddings[0]}")
def splitParargraphs(input_string):
	return input_string.split("\n\n")


def splitNewlines(input_string):
	while "\n\n" in input_string:
		input_string = input_string.replace("\n\n", "\n")
	return input_string.split("\n")


chunk_hashes = []
chunk_sections = []
for q in questions:
	entry = q_dict[q]
	hashes = []
	chunks = []
	for k, v in entry.items():
		if type(v) == list:
			for i in v:
				if type(i) == dict:
					if "chunk_hash" in i:
						hashes.append(i["chunk_hash"])
	chunk_hashes.append(hashes)


def getMostRelevant(question_embedding, chunk):
	pars = splitNewlines(chunk)

	_embeddings = m.client.models.embed_content(
		model="gemini-embedding-001",
		contents=pars,
		config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT"),
	)
	embeddings = [e.values for e in _embeddings.embeddings]
	idx = np.argmax(np.dot(embeddings, question_embedding))
	return pars[idx]


for i, e in enumerate(q_embeddings):
	# print(e)
	idx = np.argmax(np.dot(p_embeddings, e))
	most_rel_hash = list(pol[idx].keys())[0]

	if most_rel_hash not in chunk_hashes[i]:
		print("miss")
		print(f"Question: {questions[i]}")

		print(f"Most relevant section: {most_rel_hash}")
		print(f"LLMs section: {chunk_hashes[i]}")
		print()
		# demo_hash = most_rel_hash	# chunk_hashes[i][0]
		demo_hash = chunk_hashes[i][0]
		sec = ""
		for i in pol:
			if demo_hash in i:
				sec = i[demo_hash]
		if sec != "":

			most_rel_part = getMostRelevant(e, sec)
			print(most_rel_part)
		print("---")
# sec = ""
# test_hash = "a58f6449acc07a431dd2bf48b20865edf2523f3c41b57eade43cc30d4ee25f07"
# for i in pol:
# 	if test_hash in i:
# 		sec = i[test_hash]

print(sec)

# p_embeddings

miss
Question: Does the privacy policy affirm that the described practices relate to Personal Data collected when users use the company's services?
Most relevant section: 71d65acf597a239d891187f32fd89fa994a3d2b43aa14e6343ec82f546ee2622
LLMs section: ['a58f6449acc07a431dd2bf48b20865edf2523f3c41b57eade43cc30d4ee25f07']

For individuals outside the European Economic Area, United Kingdom, and Switzerland, you can read this [version⁠](/policies/row-privacy-policy/) of our Privacy Policy
---
miss
Question: Does the privacy policy affirm that the references to "we," "our," or "us" include the company and its affiliates?
Most relevant section: 7a32cf1311bbe67f865aa79a7b3c56f179f1d1f75ec50e77b78723d4a617a9ab
LLMs section: ['a58f6449acc07a431dd2bf48b20865edf2523f3c41b57eade43cc30d4ee25f07']

We at OpenAI OpCo, LLC (together with our affiliates, “OpenAI”, “we”, “our” or “us”) respect your privacy and are strongly committed to keeping secure any information we obtain from you or about you. This Pr

<!-- `"We collect personal data relating to you (\u201cPersonal Data\u201d) as follows:\n\n**Personal Data You Provide:** We collect Personal Data if you create an account to use our Services or communicate with us as follows:\n\n* *Account Information:* When you create an account with us, we will collect information associated with your account, including your name, contact information, account credentials, date of birth, payment information, and transaction history, (collectively, \u201cAccount Information\u201d).\n* *User Content:* We collect Personal Data that you provide in the input to our Services (\u201cContent\u201d), including your prompts and other content you upload, such as [files\u2060(opens in a new window)](https://help.openai.com/en/articles/8555545-file-uploads-faq), [images\u2060(opens in a new window)](https://help.openai.com/en/articles/8400551-image-inputs-for-chatgpt-faq), and [audio\u2060(opens in a new window)](https://help.openai.com/en/articles/8400625-voice-chat-faq), depending on the features you use.\n* *Communication Information*: If you communicate with us, such as via email or our pages on social media sites, we may collect Personal Data like your name, contact information, and the contents of the messages you send (\u201cCommunication Information\u201d).\n* *Other Information You Provide*: We collect other information that you may provide to us, such as when you participate in our events or surveys or provide us with information to establish your identity or age (collectively, \u201cOther Information You Provide\u201d).\n\n**Personal Data We Receive from Your Use of the Services:** When you visit, use, or interact with the Services, we receive the following information about your visit, use, or interactions (\u201cTechnical Information\u201d):\n\n* *Log Data*: We collect information that your browser or device automatically sends when you use our Services. Log data includes your Internet Protocol address, browser type and settings, the date and time of your request, and how you interact with our Services.\n* *Usage Data*: We collect information about your use of the Services, such as the types of content that you view or engage with, the features you use and the actions you take, as well as your time zone, country, the dates and times of access, user agent and version, type of computer or mobile device, and your computer connection.\n* *Device Information*: We collect information about the device you use to access the Services, such as the name of the device, operating system, device identifiers, and browser you are using. Information collected may depend on the type of device you use and its settings.\n* *Location Information:* We may determine the general area from which your device accesses our Services based on information like its IP address for security reasons and to make your product experience better, for example to protect your account by detecting unusual login activity or to provide more accurate responses. In addition, some of our Services allow you to choose to provide more precise location information from your device, such as location information from your device\u2019s GPS.\n* *Cookies and Similar Technologies*: We use cookies and similar technologies to operate and administer our Services, and improve your experience. If you use our Services without creating an account, we may store some of the information described in this policy with cookies, for example to help maintain your preferences across browsing sessions. For details about our use of cookies, please read our [Cookie Notice](/policies/cookie-policy/).\n\n**Information We Receive from Other Sources:** We receive information from our trusted partners, such as security partners, to protect against fraud, abuse, and other security threats to our Services, and from marketing vendors who provide us with information about potential customers of our business services.\n\nWe also collect information from other sources, like information that is publicly available on the internet, to develop the models that power our Services. For more information on the sources of information used to develop the models that power ChatGPT and other Services, please see [this article](/policies/how-chatgpt-and-our-foundation-models-are-developed/).`

`For individuals outside the European Economic Area, United Kingdom, and Switzerland, you can read this [version\u2060](/policies/row-privacy-policy/) of our Privacy Policy\n\nWe at OpenAI OpCo, LLC (together with our affiliates, \u201cOpenAI\u201d, \u201cwe\u201d, \u201cour\u201d or \u201cus\u201d) respect your privacy and are strongly committed to keeping secure any information we obtain from you or about you. This Privacy Policy describes our practices with respect to Personal Data that we collect from or about you when you use our website, applications, and services (collectively, \u201cServices\u201d).\n\nThis Privacy Policy does not apply to content that we process on behalf of customers of our business offerings, such as our API. Our use of that data is governed by our customer agreements covering access to and use of those offerings.\n\nFor information about how we collect and use training information to develop our language models that power ChatGPT and other Services, and your choices with respect to that information, please see [this help center article\u2060(opens in a new window)](https://help.openai.com/en/articles/7842364-how-chatgpt-and-our-language-models-are-developed).` -->

NameError: name 'e' is not defined

In [ ]:
stri = """We collect personal data relating to you (\u201cPersonal Data\u201d) as follows:\n\n**Personal Data You Provide:** We collect Personal Data if you create an account to use our Services or communicate with us as follows:\n\n* *Account Information:* When you create an account with us, we will collect information associated with your account, including your name, contact information, account credentials, date of birth, payment information, and transaction history, (collectively, \u201cAccount Information\u201d).\n* *User Content:* We collect Personal Data that you provide in the input to our Services (\u201cContent\u201d), including your prompts and other content you upload, such as [files\u2060(opens in a new window)](https://help.openai.com/en/articles/8555545-file-uploads-faq), [images\u2060(opens in a new window)](https://help.openai.com/en/articles/8400551-image-inputs-for-chatgpt-faq), and [audio\u2060(opens in a new window)](https://help.openai.com/en/articles/8400625-voice-chat-faq), depending on the features you use.\n* *Communication Information*: If you communicate with us, such as via email or our pages on social media sites, we may collect Personal Data like your name, contact information, and the contents of the messages you send (\u201cCommunication Information\u201d).\n* *Other Information You Provide*: We collect other information that you may provide to us, such as when you participate in our events or surveys or provide us with information to establish your identity or age (collectively, \u201cOther Information You Provide\u201d).\n\n**Personal Data We Receive from Your Use of the Services:** When you visit, use, or interact with the Services, we receive the following information about your visit, use, or interactions (\u201cTechnical Information\u201d):\n\n* *Log Data*: We collect information that your browser or device automatically sends when you use our Services. Log data includes your Internet Protocol address, browser type and settings, the date and time of your request, and how you interact with our Services.\n* *Usage Data*: We collect information about your use of the Services, such as the types of content that you view or engage with, the features you use and the actions you take, as well as your time zone, country, the dates and times of access, user agent and version, type of computer or mobile device, and your computer connection.\n* *Device Information*: We collect information about the device you use to access the Services, such as the name of the device, operating system, device identifiers, and browser you are using. Information collected may depend on the type of device you use and its settings.\n* *Location Information:* We may determine the general area from which your device accesses our Services based on information like its IP address for security reasons and to make your product experience better, for example to protect your account by detecting unusual login activity or to provide more accurate responses. In addition, some of our Services allow you to choose to provide more precise location information from your device, such as location information from your device\u2019s GPS.\n* *Cookies and Similar Technologies*: We use cookies and similar technologies to operate and administer our Services, and improve your experience. If you use our Services without creating an account, we may store some of the information described in this policy with cookies, for example to help maintain your preferences across browsing sessions. For details about our use of cookies, please read our [Cookie Notice](/policies/cookie-policy/).\n\n**Information We Receive from Other Sources:** We receive information from our trusted partners, such as security partners, to protect against fraud, abuse, and other security threats to our Services, and from marketing vendors who provide us with information about potential customers of our business services.\n\nWe also collect information from other sources, like information that is publicly available on the internet, to develop the models that power our Services. For more information on the sources of information used to develop the models that power ChatGPT and other Services, please see [this article](/policies/how-chatgpt-and-our-foundation-models-are-developed/)."""

print(splitParargraphs(stri))

# m = GeminiModel()


def splitParargraphs(input_string):
	return input_string.split("\n\n")


def joinParargraphs(string_array):
	return "\n\n".join(string_array)


def getEmbeddings(low_string, high_string):

	_embeddings = m.client.models.embed_content(
		model="gemini-embedding-001",
		contents=[low_string, high_string],
		config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT"),
	)
	return [_embeddings.embeddings[0].values, _embeddings.embeddings[1].values]


def getMostRelevant(question_embedding, chunk):
	chunks = halfChunk(chunk)
	if len(chunks) == 1:
		return chunks[0]
	embedding_pair = getEmbeddings()
	idx = np.argmax(np.dot(embedding_pair, [question_embedding]))
	return getMostRelevant(chunks[idx])


def halfChunk(chunk):
	split = splitParargraphs(chunk)
	if len(split) == 1:
		return chunk
	half = len(split) // 2
	chunk_start = joinParargraphs(split[:half])
	chunk_end = joinParargraphs(split[half:])
	return [chunk_start, chunk_end]

['We collect personal data relating to you (“Personal Data”) as follows:', '**Personal Data You Provide:** We collect Personal Data if you create an account to use our Services or communicate with us as follows:', '* *Account Information:* When you create an account with us, we will collect information associated with your account, including your name, contact information, account credentials, date of birth, payment information, and transaction history, (collectively, “Account Information”).\n* *User Content:* We collect Personal Data that you provide in the input to our Services (“Content”), including your prompts and other content you upload, such as [files\u2060(opens in a new window)](https://help.openai.com/en/articles/8555545-file-uploads-faq), [images\u2060(opens in a new window)](https://help.openai.com/en/articles/8400551-image-inputs-for-chatgpt-faq), and [audio\u2060(opens in a new window)](https://help.openai.com/en/articles/8400625-voice-chat-faq), depending on the feature

'We collect personal data relating to you (“Personal Data”) as follows:\n\n**Personal Data You Provide:** We collect Personal Data if you create an account to use our Services or communicate with us as follows:\n\n* *Account Information:* When you create an account with us, we will collect information associated with your account, including your name, contact information, account credentials, date of birth, payment information, and transaction history, (collectively, “Account Information”).\n* *User Content:* We collect Personal Data that you provide in the input to our Services (“Content”), including your prompts and other content you upload, such as [files\u2060(opens in a new window)](https://help.openai.com/en/articles/8555545-file-uploads-faq), [images\u2060(opens in a new window)](https://help.openai.com/en/articles/8400551-image-inputs-for-chatgpt-faq), and [audio\u2060(opens in a new window)](https://help.openai.com/en/articles/8400625-voice-chat-faq), depending on the features

In [ ]:
a = "asfjasfba"
a[:5]

'asfja'

In [4]:
a[5:]

'sfba'

In [4]:
from policy_analyser import loadQuestions

qs = loadQuestions()
for k in qs:
	print(k)

Does the privacy policy affirm that the policy explains how the company collects personal data?
Does the privacy policy affirm that the policy explains how the company uses personal data?
Does the privacy policy affirm that the policy explains how the company discloses personal data?
Does the privacy policy affirm that the policy explains how the company processes personal data?
Does the privacy policy affirm that the policy applies to instances where the company acts as a data controller?
Does the privacy policy affirm that the policy applies when users interact with the company's products as a consumer for personal use?
Does the privacy policy affirm that the policy applies when the company provides commercial customers and their end users access to its commercial products?
Does the privacy policy affirm that the policy does not apply when the company acts as a data processor?
Does the privacy policy affirm that the commercial customer is the controller when the company processes dat

In [ ]:
# AnalysisProcessor.extractContent(
# 	"https://cdn.deepseek.com/policies/en-US/deepseek-privacy-policy.html"
# )

- Remove questions that contain flag words like OpenAI and Anthropic

Questions should contain one unique affirmation and should be as consice as possible. For example instead of `Does the privacy policy affirm that account creation leads to the collection of the user's name, contact information, account credentials, date of birth, payment information, and transaction history?`, you should generate 6 questions:
```
Does the privacy policy affirm that account creation leads to the collection of the user's name?
Does the privacy policy affirm that account creation leads to the collection of contact information?
Does the privacy policy affirm that account creation leads to the collection of account credentials?
Does the privacy policy affirm that account creation leads to the collection of payment information?
Does the privacy policy affirm that account creation leads to the collection of transaction history?
```

Each affirmation must not be self referential to the document structure, for example: `Does the privacy policy affirm that Section 11 contains provisions that apply specifically to users located in Canada?` is an invalid question, since it is to do with the document itself. We should omit questions like that unless the relevent text is in the document, then we should similarly proceed with direct affirmation questions, to do with the specific information.

**Atomicity Constraint:** Each generated question must contain only one unique, verifiable affirmation (one single fact, policy, or commitment). Questions must be concise. Do not combine multiple facts, data points, or policy clauses into a single question.

For example, instead of the multi-fact question:
`Does the policy affirm that account creation leads to the collection of the user's name, contact information, account credentials, date of birth, payment information, and transaction history?`

You must generate six separate, atomic questions, such as:
```
Does the privacy policy affirm that account creation leads to the collection of the user's name?
Does the privacy policy affirm that account creation leads to the collection of contact information?
Does the privacy policy affirm that account creation leads to the collection of account credentials?
Does the privacy policy affirm that account creation leads to the collection of payment information?
Does the privacy policy affirm that account creation leads to the collection of transaction history?
```


Affirmation questions must pertain directly to the policies, actions, disclosures, or specific commitments described within this text snippet. Questions must not be self-referential to the document's structure, chapter titles, internal cross-references, or numbering scheme, **unless that structural element is the primary subject of the question and explicitly stated in the provided text.**

For example, questions about implied structure or location (e.g., `Does the privacy policy affirm that Section 11 contains provisions that apply specifically to users located in Canada?`) are invalid, as Section 11 is not present.

Instead, focus only on the specific policy dictates, such as: `Does the policy affirm that data for Canadian residents may be transferred to jurisdictions where data protection laws are less stringent?`

Does the privacy policy affirm that account creation leads to the collection of the user's name, contact information, account credentials, date of birth, payment information, and transaction history?

Does the privacy policy affirm that OpenAI OpCo, LLC is the controller responsible for processing Personal Data of users living in the UK?

Does the privacy policy affirm that Section 11 contains provisions that apply specifically to users located in Canada?

Does the privacy policy affirm that Anthropic Ireland, Limited is the data controller responsible for personal data of users who live in the European Region (EEA, UK, or Switzerland)?

In [ ]:
from policy_analyser import loadQuestions

qs = loadQuestions()
for k in qs:
	print(k)

In [ ]:
import numpy as np
from model_management import GeminiModel

# question_1 = "Does the privacy policy affirm that personal data transfers are automatically consented to by using the service?"
# question_1_prime = "Does the privacy policy affirm that user data transfers are automatically consented to by using the service?"
# question_2 = "Does the privacy policy affirm that personal data processing is automatically consented to by using the service?"

question_1 = "Does the privacy policy affirm that personal data processing is automatically consented to by using the service?"
question_1_prime = "Does the privacy policy affirm that processing of personal data is automatically consented to by using the service?"
question_2 = "Does the privacy policy affirm that personal data transfers are automatically consented to by using the service?"

preprocess_question_1 = GeminiModel.preprocessStatement(question_1)
preprocess_question_1_prime = GeminiModel.preprocessStatement(question_1_prime)
preprocess_question_2 = GeminiModel.preprocessStatement(question_2)


preprocess_question_1_alt = GeminiModel.preprocessStatement_alt(question_1)
preprocess_question_1_prime_alt = GeminiModel.preprocessStatement_alt(question_1_prime)
preprocess_question_2_alt = GeminiModel.preprocessStatement_alt(question_2)

original = [question_1, question_1_prime, question_2]

preprocess_main = [
	preprocess_question_1,
	preprocess_question_1_prime,
	preprocess_question_2,
]
preprocess_alt = [
	preprocess_question_1_alt,
	preprocess_question_1_prime_alt,
	preprocess_question_2_alt,
]


task_types = [
	"SEMANTIC_SIMILARITY",
	"QUESTION_ANSWERING",
	"CLASSIFICATION",
	"CLUSTERING",
	"RETRIEVAL_DOCUMENT",
	"RETRIEVAL_QUERY",
	"FACT_VERIFICATION",
]
# add Mahalanobis later for entire dataset


def pQs():
	model = GeminiModel()

	def processEmbeddings(q_set):
		embs = {}
		for i in task_types:
			emb_1 = model.getSemanticEmbedding(q_set[0], task_type=i)
			emb_1_prime = model.getSemanticEmbedding(q_set[1], task_type=i)
			emb_2 = model.getSemanticEmbedding(q_set[2], task_type=i)
			embs.update({i: [emb_1, emb_1_prime, emb_2]})
		return embs

	return {
		"Original": processEmbeddings(original),
		"Main": processEmbeddings(preprocess_main),
		"Alt": processEmbeddings(preprocess_alt),
	}


embedding_sets = pQs()

In [ ]:
import re


cleaned_result = cleanString(input_text)

print(f"Original String: '{input_text}'")
print(f"Cleaned String:  '{cleaned_result}'")

In [ ]:
def processQsets(embedding_sets, similarity="cosine"):

	similarity_processors = {
		"cosine": lambda emb_a, emb_b: np.dot(emb_a, emb_b)
		/ (np.linalg.norm(emb_a) * np.linalg.norm(emb_b)),
		"l1": lambda emb_a, emb_b: np.linalg.norm(np.array(emb_a) - np.array(emb_b), ord=1),
		"l2": lambda emb_a, emb_b: np.linalg.norm(np.array(emb_a) - np.array(emb_b), ord=2),
		"dot": lambda emb_a, emb_b: np.dot(emb_a, emb_b),
	}
	if similarity not in similarity:
		similarity = "cosine"

	similarityProcessor = similarity_processors[similarity]

	data = {}

	for k, v in embedding_sets.items():
		_data = {}
		for l, w in v.items():
			[emb_1, emb_1_prime, emb_2] = w
			semantically_identical = similarityProcessor(emb_1, emb_1_prime)
			lexically_similar = similarityProcessor(emb_1, emb_2)
			delta = semantically_identical - lexically_similar
			_data.update(
				{
					l: {
						"semantically_identical": semantically_identical,
						"lexically_similar": lexically_similar,
						"delta": delta,
					}
				}
			)
		data.update({k: _data})
	return data

In [ ]:
import pandas as pd

data = processQsets(embedding_sets, "dot")


reformatted_data = {
	(outer_key, inner_key): values
	for outer_key, inner_dict in data.items()
	for inner_key, values in inner_dict.items()
}

df = pd.DataFrame.from_dict(reformatted_data, orient="index")

df.index.names = ["Setting", "Task Type"]
pd.set_option("display.width", 1000)
print(df)

df_sorted = df.sort_values(by="delta", ascending=False)
print("--- ordered ---")
print(df_sorted)

In [ ]:
# data = processQsets("dot")

In [ ]:
a = "f"

b = {"f": 1}

a in b

# Cosine
                              semantically_identical  lexically_similar     delta
Setting  Task Type                                                               
Original SEMANTIC_SIMILARITY                0.997794           0.970933  0.026861
         QUESTION_ANSWERING                 0.984825           0.898482  0.086343
         CLASSIFICATION                     0.987050           0.929606  0.057444
         CLUSTERING                         0.995677           0.966805  0.028872
         RETRIEVAL_DOCUMENT                 0.995890           0.957301  0.038589
         RETRIEVAL_QUERY                    0.985738           0.910084  0.075654
         FACT_VERIFICATION                  0.986830           0.909675  0.077155
Main     SEMANTIC_SIMILARITY                0.997109           0.959077  0.038032
         QUESTION_ANSWERING                 0.984425           0.899066  0.085359
         CLASSIFICATION                     0.980436           0.914609  0.065828
         CLUSTERING                         0.993968           0.954111  0.039857
         RETRIEVAL_DOCUMENT                 0.995030           0.953517  0.041513
         RETRIEVAL_QUERY                    0.972311           0.889050  0.083261
         FACT_VERIFICATION                  0.981177           0.892366  0.088811
Alt      SEMANTIC_SIMILARITY                0.997718           0.973772  0.023947
         QUESTION_ANSWERING                 0.985752           0.916921  0.068831
         CLASSIFICATION                     0.988074           0.960987  0.027087
         CLUSTERING                         0.994682           0.980497  0.014185
         RETRIEVAL_DOCUMENT                 0.994648           0.964293  0.030356
         RETRIEVAL_QUERY                    0.981498           0.922341  0.059157
         FACT_VERIFICATION                  0.985763           0.924770  0.060994
--- ordered ---
                              semantically_identical  lexically_similar     delta
Setting  Task Type                                                               
Main     FACT_VERIFICATION                  0.981177           0.892366  0.088811
Original QUESTION_ANSWERING                 0.984825           0.898482  0.086343
Main     QUESTION_ANSWERING                 0.984425           0.899066  0.085359
         RETRIEVAL_QUERY                    0.972311           0.889050  0.083261
Original FACT_VERIFICATION                  0.986830           0.909675  0.077155
         RETRIEVAL_QUERY                    0.985738           0.910084  0.075654
Alt      QUESTION_ANSWERING                 0.985752           0.916921  0.068831
Main     CLASSIFICATION                     0.980436           0.914609  0.065828
Alt      FACT_VERIFICATION                  0.985763           0.924770  0.060994
         RETRIEVAL_QUERY                    0.981498           0.922341  0.059157
Original CLASSIFICATION                     0.987050           0.929606  0.057444
Main     RETRIEVAL_DOCUMENT                 0.995030           0.953517  0.041513
         CLUSTERING                         0.993968           0.954111  0.039857
Original RETRIEVAL_DOCUMENT                 0.995890           0.957301  0.038589
Main     SEMANTIC_SIMILARITY                0.997109           0.959077  0.038032
Alt      RETRIEVAL_DOCUMENT                 0.994648           0.964293  0.030356
Original CLUSTERING                         0.995677           0.966805  0.028872
Alt      CLASSIFICATION                     0.988074           0.960987  0.027087
Original SEMANTIC_SIMILARITY                0.997794           0.970933  0.026861
Alt      SEMANTIC_SIMILARITY                0.997718           0.973772  0.023947
         CLUSTERING                         0.994682           0.980497  0.014185
# L1

                              semantically_identical  lexically_similar      delta
Setting  Task Type                                                                
Original SEMANTIC_SIMILARITY                2.901578          10.519026  -7.617447
         QUESTION_ANSWERING                 7.580808          19.704753 -12.123945
         CLASSIFICATION                     7.063572          16.405605  -9.342033
         CLUSTERING                         4.078835          11.330158  -7.251323
         RETRIEVAL_DOCUMENT                 3.947688          12.837708  -8.890020
         RETRIEVAL_QUERY                    7.414612          18.483660 -11.069048
         FACT_VERIFICATION                  7.067743          18.594226 -11.526483
Main     SEMANTIC_SIMILARITY                3.354251          12.554120  -9.199869
         QUESTION_ANSWERING                 7.711456          19.678749 -11.967293
         CLASSIFICATION                     8.656327          17.951024  -9.294697
         CLUSTERING                         4.820952          13.303891  -8.482940
         RETRIEVAL_DOCUMENT                 4.352501          13.225239  -8.872738
         RETRIEVAL_QUERY                   10.421610          20.682896 -10.261286
         FACT_VERIFICATION                  8.483453          20.292357 -11.808904
Alt      SEMANTIC_SIMILARITY                2.948793           9.970297  -7.021504
         QUESTION_ANSWERING                 7.444575          17.861544 -10.416970
         CLASSIFICATION                     6.806107          12.145342  -5.339235
         CLUSTERING                         4.469010           8.674672  -4.205662
         RETRIEVAL_DOCUMENT                 4.559093          11.740893  -7.181800
         RETRIEVAL_QUERY                    8.457980          17.229176  -8.771196
         FACT_VERIFICATION                  7.403331          16.978476  -9.575144
--- ordered ---
                              semantically_identical  lexically_similar      delta
Setting  Task Type                                                                
Alt      CLUSTERING                         4.469010           8.674672  -4.205662
         CLASSIFICATION                     6.806107          12.145342  -5.339235
         SEMANTIC_SIMILARITY                2.948793           9.970297  -7.021504
         RETRIEVAL_DOCUMENT                 4.559093          11.740893  -7.181800
Original CLUSTERING                         4.078835          11.330158  -7.251323
         SEMANTIC_SIMILARITY                2.901578          10.519026  -7.617447
Main     CLUSTERING                         4.820952          13.303891  -8.482940
Alt      RETRIEVAL_QUERY                    8.457980          17.229176  -8.771196
Main     RETRIEVAL_DOCUMENT                 4.352501          13.225239  -8.872738
Original RETRIEVAL_DOCUMENT                 3.947688          12.837708  -8.890020
Main     SEMANTIC_SIMILARITY                3.354251          12.554120  -9.199869
         CLASSIFICATION                     8.656327          17.951024  -9.294697
Original CLASSIFICATION                     7.063572          16.405605  -9.342033
Alt      FACT_VERIFICATION                  7.403331          16.978476  -9.575144
Main     RETRIEVAL_QUERY                   10.421610          20.682896 -10.261286
Alt      QUESTION_ANSWERING                 7.444575          17.861544 -10.416970
Original RETRIEVAL_QUERY                    7.414612          18.483660 -11.069048
         FACT_VERIFICATION                  7.067743          18.594226 -11.526483
Main     FACT_VERIFICATION                  8.483453          20.292357 -11.808904
         QUESTION_ANSWERING                 7.711456          19.678749 -11.967293
Original QUESTION_ANSWERING                 7.580808          19.704753 -12.123945


# L2
                              semantically_identical  lexically_similar     delta
Setting  Task Type                                                               
Original SEMANTIC_SIMILARITY                0.066429           0.241111 -0.174682
         QUESTION_ANSWERING                 0.174212           0.450595 -0.276384
         CLASSIFICATION                     0.160934           0.375216 -0.214282
         CLUSTERING                         0.092979           0.257662 -0.164683
         RETRIEVAL_DOCUMENT                 0.090664           0.292229 -0.201565
         RETRIEVAL_QUERY                    0.168892           0.424066 -0.255173
         FACT_VERIFICATION                  0.162297           0.425030 -0.262733
Main     SEMANTIC_SIMILARITY                0.076040           0.286086 -0.210046
         QUESTION_ANSWERING                 0.176492           0.449298 -0.272805
         CLASSIFICATION                     0.197806           0.413258 -0.215452
         CLUSTERING                         0.109840           0.302949 -0.193110
         RETRIEVAL_DOCUMENT                 0.099696           0.304902 -0.205207
         RETRIEVAL_QUERY                    0.235327           0.471062 -0.235736
         FACT_VERIFICATION                  0.194027           0.463970 -0.269943
Alt      SEMANTIC_SIMILARITY                0.067551           0.229035 -0.161484
         QUESTION_ANSWERING                 0.168808           0.407624 -0.238816
         CLASSIFICATION                     0.154444           0.279332 -0.124888
         CLUSTERING                         0.103130           0.197499 -0.094369
         RETRIEVAL_DOCUMENT                 0.103457           0.267235 -0.163779
         RETRIEVAL_QUERY                    0.192363           0.394104 -0.201742
         FACT_VERIFICATION                  0.168742           0.387893 -0.219151
--- ordered ---
                              semantically_identical  lexically_similar     delta
Setting  Task Type                                                               
Alt      CLUSTERING                         0.103130           0.197499 -0.094369
         CLASSIFICATION                     0.154444           0.279332 -0.124888
         SEMANTIC_SIMILARITY                0.067551           0.229035 -0.161484
         RETRIEVAL_DOCUMENT                 0.103457           0.267235 -0.163779
Original CLUSTERING                         0.092979           0.257662 -0.164683
         SEMANTIC_SIMILARITY                0.066429           0.241111 -0.174682
Main     CLUSTERING                         0.109840           0.302949 -0.193110
Original RETRIEVAL_DOCUMENT                 0.090664           0.292229 -0.201565
Alt      RETRIEVAL_QUERY                    0.192363           0.394104 -0.201742
Main     RETRIEVAL_DOCUMENT                 0.099696           0.304902 -0.205207
         SEMANTIC_SIMILARITY                0.076040           0.286086 -0.210046
Original CLASSIFICATION                     0.160934           0.375216 -0.214282
Main     CLASSIFICATION                     0.197806           0.413258 -0.215452
Alt      FACT_VERIFICATION                  0.168742           0.387893 -0.219151
Main     RETRIEVAL_QUERY                    0.235327           0.471062 -0.235736
Alt      QUESTION_ANSWERING                 0.168808           0.407624 -0.238816
Original RETRIEVAL_QUERY                    0.168892           0.424066 -0.255173
         FACT_VERIFICATION                  0.162297           0.425030 -0.262733
Main     FACT_VERIFICATION                  0.194027           0.463970 -0.269943
         QUESTION_ANSWERING                 0.176492           0.449298 -0.272805
Original QUESTION_ANSWERING                 0.174212           0.450595 -0.276384


In [ ]:
import threading


def update_json_safely(lock: threading.Lock, update_dict, debug: bool = False):

	with lock:

		with open(ALL_EXAMS_DATA, "r", encoding="utf-8") as f:
			data = json.load(f)
			if not isinstance(data, dict):

				data = {}	# Ensure we have a dictionary to work with

	data.update(update_dict)
	for k, v in update_dict.items():
		if k not in data:
			print(f"Updating json with {k}")

		# Write the entire modified structure back to the file
	with lock:

		with open(ALL_EXAMS_DATA, "w", encoding="utf-8") as f:
			json.dump(data, f, ensure_ascii=False, indent=4)

		# if debug:
		#     print(
		#         f"[Thread {threading.get_ident()}] Lock released for {filename}."
		#     )
	return data